In [18]:
import numpy as np
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, AutoModelForSequenceClassification, pipeline, trainer
from transformers import Trainer, TrainingArguments



In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelname = "hfl/chinese-xlnet-base"
tokenizer = AutoTokenizer.from_pretrained(modelname)

max_len = 300


In [1]:


model = AutoModelForSequenceClassification.from_pretrained(modelname)

# raw_datasets = load_dataset("imdb")

Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [3]:
def tokenize_function(examples):
    return tokenizer(examples['text'], max_length=max_len, padding='max_length', return_tensors = 'pt')

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [4]:
chn_train = load_dataset('seamew/ChnSentiCorp')
# BATCH=32
# MAX_SEQLEN=300
# LR=5e-5
# EPOCH=10

Using custom data configuration default
Reusing dataset chn_senti_corp (/home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:

def process_data(data):
    # max length is about 306..., truncation and padding are needed
    return tokenizer(data["text"], max_length=max_len, padding='max_length', truncation=True)


In [6]:
tokenized_datasets = chn_train.map(process_data)

Loading cached processed dataset at /home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-6aafad47b22fbe0f.arrow


  0%|          | 0/1200 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-45176dcc9c061a09.arrow


In [7]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

Loading cached shuffled indices for dataset at /home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-a59dd62e4d1dd04b.arrow
Loading cached shuffled indices for dataset at /home/kesong/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-34c7cfa246b4ef5d.arrow


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=2)


In [21]:

num_epoch = 3
per_device_batch = 48

training_args = TrainingArguments(
  'finetune_trainer', 
  evaluation_strategy='epoch',
  num_train_epochs=num_epoch, 
  per_device_train_batch_size=per_device_batch, 
  per_device_eval_batch_size=per_device_batch
)

In [11]:
model.to(device)
trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [12]:
# print(fine_tune_train.columns)
torch.cuda.empty_cache()
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 9600
  Num Epochs = 3
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 300
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.231794
2,No log,0.182594
3,No log,0.181102


The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96
The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96
The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1200
  Batch size = 96


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=0.17257573445638022, metrics={'train_runtime': 597.9458, 'train_samples_per_second': 48.165, 'train_steps_per_second': 0.502, 'total_flos': 4807354717440000.0, 'train_loss': 0.17257573445638022, 'epoch': 3.0})

In [3]:
import os
os.makedirs("./xlnet-finetuned-1")

FileExistsError: [Errno 17] File exists: './xlnet-finetuned-1'

In [14]:
model.save_pretrained("./xlnet-finetuned-1")

Configuration saved in ./xlnet-finetuned-1/config.json
Model weights saved in ./xlnet-finetuned-1/pytorch_model.bin


In [4]:
model = AutoModelForSequenceClassification.from_pretrained("./xlnet-finetuned-1")

In [5]:
danmuku = pd.read_csv('test_dataframe.csv', nrows=200)
danmuku.shape

(200, 3)

In [6]:
danmuku['label'] = danmuku['label'].apply(lambda x: 1 if x < 0 else x)
for i in danmuku['label']:
    if i < 0:
        print(-1)

In [7]:
danmuku = danmuku.loc[:, ~danmuku.columns.str.contains('^Unnamed')]


In [8]:
from datasets import Dataset
danmuku_dataset = Dataset.from_pandas(danmuku)


In [15]:
tokenized_danmuku_dataset = danmuku_dataset.map(process_data)


  0%|          | 0/200 [00:00<?, ?ex/s]

In [16]:
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
testor = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_danmuku_dataset.shuffle(seed=42).select(range(160)),
    eval_dataset=tokenized_danmuku_dataset.shuffle(seed=42).select(range(160, 200)),
    compute_metrics=compute_metrics,
)

In [38]:
torch.cuda.empty_cache()

In [23]:
model.to(device)
testor.train()

The following columns in the training set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 160
  Num Epochs = 3
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 1
  Total optimization steps = 6
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.992033,0.675000
2,No log,0.586243,0.725000
3,No log,0.622488,0.725000


The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather

TrainOutput(global_step=6, training_loss=1.1162512302398682, metrics={'train_runtime': 13.4469, 'train_samples_per_second': 35.696, 'train_steps_per_second': 0.446, 'total_flos': 80122578624000.0, 'train_loss': 1.1162512302398682, 'epoch': 3.0})

In [24]:
testor.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 96
/home/kesong/.conda/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6224875450134277,
 'eval_accuracy': 0.725,
 'eval_runtime': 0.4654,
 'eval_samples_per_second': 85.956,
 'eval_steps_per_second': 2.149,
 'epoch': 3.0}

In [73]:
small_train_dataset[0]['attention_mask']


[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
